<a href="https://colab.research.google.com/github/rogersirius2023/Atvidades/blob/main/Clustering_Semana4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Agrupamento Hierárquico (Hierarchical Clustering)
Há duas categorias de agrupamento hierárquico de dados: agrupamento aglomerativo e agrupamento divisivo. Vamos entender como essas duas estratégias de agrupamento funcionam.

Aglomerativo: Esta é uma estratégia de agrupamento que usa a ideia de agrupar elementos "de baixo para cima", isto é, começa supondo que cada ponto é um grupo por si só e então vai unindo os grupos mais similares entre si para transformá-los em novos grupos maiores. Então começa-se com a maior quantidade possível de grupos (quantidade de observações nos dados) e termina com um único grupo.

Divisivo: Esta é uma estratégia de agrupamento que usa a ideia de agrupar "de cima para baixo", sito é, começa com um único grupo contendo todas as observações e então vai subdividindo esse grupo em grupos menores de acordo com as dissimilares. Aqui podemos entender como os grupos se relacionam e o quanto são diferentes uns dos outros.

Demonstraremos aqui o uso de técnicas de agrupamento hierárquico aglomerativo, que são mais utilizadas - e uma das principais razões se deve ao custo computacional, que é muito maior nas técnicas divisivas se comparado às técnicas aglomerativas.

Para ambos os casos, os dendrogramas são uma excelente ferramenta gráfica para visualizar os as relações entre elementos e grupos.

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')
df = pd.read_excel('/content/drive/MyDrive/Atividades Siruis/Fonte dos Dados.xlsx')
df.head()

Vamos selecionar apenas as variáveis % de pobres e Taxa de desemprego, assim como fizemos com o KMeans, para ter uma comparação das técnicas. Também vamos agrupar os dados por estado para trabalhar com uma quantidade menor de dados.

Esse passo do agrupamento é importante, pois técnicas de agrupamento podem ser computacionalmente custosas para grandes conjuntos de dados, por conta da grande quantidade de distâncias que são calculadas para cada observação, conforme veremos a seguir.

In [ ]:
df2 = df[['estado','PIB Municipal','% de pobres', 'IDH',
          'Média Salarial','Taxa de desemprego']]          
df2 = df2.groupby(by='estado').mean()
df2 = df2.reset_index()
df2.head()      

Devemos sempre lembrar que técnicas que envolvem cálculo de distância entre observações devem ter os dados na mesma escala. Por isso vamos usar o StandardScaler, para mais detalhes veja a documentação.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_scaled  = df2.copy()

df_scaled.drop('estado', inplace=True, axis=1)


df_scaled[df_scaled.columns] = scaler.fit_transform(df_scaled)


df_scaled['estado'] = df2['estado']

df_scaled.head()

Agrupamento Hierárquico Aglomerativo (com Scipy)
Para este tipo de agrupamento podemos obter uma visualização interessante mostrando a relação entre os grupos: o dendograma.

Para isso vamos importar as funções linkage e dendrogram da biblioteca SciPy

Para mais informações, consulte linkage e dendrogram

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram

Para fazer o dendograma, primeiramente precisamos das distâncias, que é obtida com o uso da função linkage.

In [ ]:
distancias = linkage(df_scaled[['Taxa de desemprego','% de pobres']],
                     method='ward',
                     metric='euclidean')

Podemos usar a função dendogram para visualizar o dendograma.

In [ ]:
import seaborn as sns
sns.set(style='white', rc={'figure.figsize':(12,5)})

dn = dendrogram(distancias,
                leaf_font_size=9,
                orientation='right',
                labels=df_scaled['estado'].values)

Analisando o dendograma podemos ver que se a distância variar de 4 a 8, a quantidade de grupos não varia, continuamos com 2 grupos. Isto indica que existem dois grupos bem distintos (distantes). Enquanto que se mudássemos nossa distância de 0.5 para 1, a quantidade grupos mudaria muito, indicando que vários desses grupos seriam muito similares (estariam muito próximos).

A conclusão é que a quantidade ideal de grupos deve estar associada à uma quantidade fixa de grupos para grandes variações da distância. Podemos analisar o dendograma para escolher a quantidade de grupos de acordo com o que precisamos, levando em consideração as similaridades entre os grupos.

Podemos também utilizar o dendograma e a função fcluster da biblioteca SciPy para automatizar esse processo.

In [ ]:
from scipy.cluster.hierarchy import fcluster
max_d = 3
grupos = fcluster(distancias, t=max_d, criterion='distance')
grupos

In [ ]:
grupos = fcluster(distancias, t=2, criterion='maxclust')
grupos

Vamos fazer um comparativo dos grupos obtidos através das duas técnicas (agrupamento hierárquico do algoritmo KMeans), representadas respectivamente pelo dendrograma e pelo gráfico de dispersão.

In [ ]:
# importando a biblioteca matplotlib
import matplotlib.pyplot as plt

sns.set(style='white', rc={'figure.figsize':(12,8)})
fig, axs = plt.subplots(ncols=1, nrows=2)

dn = dendrogram(distancias,
                leaf_rotation=0,
                leaf_font_size=9,
                orientation='right',
                labels=df_scaled['estado'].values,
                ax=axs[0])

# criando o gráfico de dispersão
sns.scatterplot(data=df2, x='Taxa de desemprego', y='% de pobres',
                hue=grupos, palette='tab10', ax=axs[1]);

# criando listas que serviram de apoio para anotar a sigla de cada estado no gráfico
desemp = df2['Taxa de desemprego']
pobres = df2['% de pobres']
estados = ['AC','AL','AP','AM','BA','CE','DF','ES','GO','MA','MT','MS','MG',
           'PR','PB','PA','PE','PI','RN','RS','RJ','RO','RR','SC','SE','SP','TO']

# anotando os 'nomes' de cada ponto
for i, estado in enumerate(estados):
    plt.annotate(estado, (desemp[i], pobres[i]) );

Podemos ver que obtivemos os mesmo grupos, de forma que não parece haver vantagem em usar o agrupamento hierárquico ou o KMeans. Mas, e se tivéssemos 4 grupos?

Outra pergunta que poderíamos nos fazer é: quais grupos são mais similares entre si?

Agora fica mais difícil responder isso apenas com base no gráfico de dispersão e na separação do KMeans, mas podemos ter uma ideia dessa similaridade observando o dendrograma.

Também ficaria difícil visualizar essas relações se tivéssemos mais de 2 variáveis, pois teríamos que visualizar a dispersão dos pontos em um espaço multidimensional.

In [ ]:
grupos = fcluster(distancias, t=4, criterion='maxclust')

sns.set(style='white', rc={'figure.figsize':(10,7)})
fig, axs = plt.subplots(ncols=1, nrows=2)

dn = dendrogram(distancias,
                leaf_rotation=0,
                leaf_font_size=9,
                orientation='right',
                labels=df_scaled['estado'].values,
                ax=axs[0])

# criando o gráfico de dispersão
sns.scatterplot(data=df2, x='Taxa de desemprego', y='% de pobres',
                hue=grupos, palette='tab10', ax=axs[1]);

# criando listas que serviram de apoio para anotar a sigla de cada estado no gráfico
desemp = df2['Taxa de desemprego']
pobres = df2['% de pobres']
estados = ['AC','AL','AP','AM','BA','CE','DF','ES','GO','MA','MT','MS','MG',
           'PR','PB','PA','PE','PI','RN','RS','RJ','RO','RR','SC','SE','SP','TO']

# importando a biblioteca matplotlib
import matplotlib.pyplot as plt

# anotando os 'nomes' de cada ponto
for i, estado in enumerate(estados):
    plt.annotate(estado, (desemp[i], pobres[i]) );

Análise dos Grupos
Vamos agrupar os dados por grupos e entender melhor como são as características de cada grupo, a fim de identificar e entender melhor cada grupo.

In [ ]:
df2['grupo'] = grupos
df2.head()

Podemos ver que o PIB municipal do grupo 0 é bem maior que dos demais; a porcentagem de pobres do grupo 3 é muito menor que nos demais; o IDH é relativamente comparavável entre os grupos, assim como a média salarial; e a taxa de desemprego é bem menor no grupo 3.

In [ ]:
# separando os grupos
grupo1 = df2[df2['grupo'] == 1]
grupo2 = df2[df2['grupo'] == 2]
grupo3 = df2[df2['grupo'] == 3]
grupo4 = df2[df2['grupo'] == 4]

In [ ]:
# estatísticas descritivas do grupo 1
grupo1.describe()

In [ ]:
# características do grupo 1
grupo1_lista = ['75% possui menos de 4.4e08',
                '75% possui menos de 15%',
                '75% possui mais de 6.4']

In [ ]:
# estatísticas descritivas do grupo 2
grupo2.describe()

In [ ]:
# características do grupo 2
grupo2_lista = ['75% possui menos de 4.8e08',
                '75% possui mais de 38%',
                '75% possui mais de 9.3']

In [ ]:
# estatísticas descritivas do grupo 3
grupo3.describe()

In [ ]:
# características do grupo 3
grupo3_lista = ['75% possui menos de 4.7e08',
                '75% possui mais de 41.8%',
                '75% possui mais de 7.2']

In [ ]:
# estatísticas descritivas do grupo 4
grupo4.describe()

In [ ]:
# características do grupo 4
grupo4_lista = ['75% mais de 5e08',
                '75% possui menos de 8.3%',
                '75% possui menos de 2.8']

In [ ]:
dicio_grupos = {'grupo 1':grupo1_lista,
                'grupo 2':grupo2_lista,
                'grupo 3':grupo3_lista,
                'grupo 4':grupo4_lista}

In [ ]:
personas = pd.DataFrame(dicio_grupos, index=['PIB Municipal','% de pobres','Taxa desemprego'])
personas

Uma maneira mais interessante de ver as características do grupos é através do boxplot.

In [ ]:
sns.set(style='white', rc={'figure.figsize':(10,5)})

# boxplot da variável PIB Minucipal
sns.boxplot(data=df2, x='grupo', y='PIB Municipal');

Podemos ver que há um outlier no grupo 2, atrapalhando o entendimento deste e dos demasi grupos, vamos identificá-lo e removê-lo.

In [ ]:
# observado o DataFrame
df2

Olhando a coluna PIB Municipal, podemos ver que o Distrito Federal tem uma valor muito maior que os demais, e portanto este é o outlier que aparece no boxplot.

In [ ]:
# removendo o outlier
df2.drop([6], axis=0, inplace=True)
df2

Podemos observar que o índice 6 não está mais presente. Agora podemos refazer o boxplot para ter melhor entendimento da variável.

In [ ]:
# boxplot da variável PIB Minucipal
sns.boxplot(data=df2, x='grupo', y='PIB Municipal');

In [ ]:
# boxplot da variável Taxa de desemprego
sns.boxplot(data=df2, x='grupo', y='Taxa de desemprego');

In [ ]:
# boxplot da variável '% de pobres'
sns.boxplot(data=df2, x='grupo', y='% de pobres');

Podemos então considerar que:

O grupo 1 é o grupo que apresenta PIB municipal baixo, baixa taxa de desemprego e baixa porcentagem de pobres (Estados pobres com população rica e empregada).

O grupo 2 é o grupo que possui PIB municipal pouco mais elevado (apesar do outlier) baixa porcentagem de pobres e moderada taxa de desemprego. (Estados ricos com alto emprego e população rica)

O grupo 3 é o grupo e o grupo que apresenta PIB municipal moderado, taxa de desemprego moderada e alta porcentagem de pobres. (Estados com moderado desemprego e população pobre)

O grupo 4 é o grupo que apresenta maior taxa de desemprego, alta porcentagem de pobres e baixo PIB municipal. (Estados pobres com alto desemprego e população pobre)

A análise acima pode ser mais interessante do que a tabela personas, embora as duas sejam complementares. Pode-se perceber que o grupo 1 é formado por estados pobres cuja população é rica; o grupo 2 é composto por estados ricos em que a população empregada ganha melhor; o grupo 3 é estados nem ricos nem pobres com população pobre; o grupo 4 é composto por estados pobres com população ganhando mal.

Essa identificação seria muito importante para nortear programas governamentais.